# MinHashing all the things: a quick analysis of MAG search results

Companion notebook for https://blog.luizirber.org/2020/07/23/mag-results/

## Preparing some metadata

In [2]:
!snakemake -j1

Building DAG of jobs...
Nothing to be done.
Complete log: /home/luizirber/work/sourmash-bio/2020-07-22-mag-search/.snakemake/log/2020-07-24T092049.591934.snakemake.log


## Loading the data

In [3]:
import re

from IPython.display import HTML
import pandas as pd
import numpy as np

data = pd.read_table("results.csv", sep=",", names=["MAG", "metagenome", "containment"])

# These MAGs come from TARA Oceans, and some of them are in Parks 8k, so let's remove them from results
tara_metag = pd.read_table("inputs/tara_runinfo.csv", sep=",", header=0, usecols=["Run"])
parks_8k = pd.read_table("inputs/parks_runinfo.csv", sep=",", header=0, usecols=["Run"])

# Fix names so it's easier to query
data['MAG'] = data['MAG'].str.replace(r"'(?P<id>.*)'", lambda m: m.group("id"))
data['metagenome'] = data['metagenome'].str.replace(r".*/(?P<id>.*).sig.*", lambda m: m.group("id"))

# Actually remove TARA and Parks
to_keep = set(data['metagenome'].values).difference(set(tara_metag["Run"].values))
to_keep = to_keep.difference(set(parks_8k["Run"].values))
filtered = data[data['metagenome'].isin(to_keep)]
print(filtered[filtered['containment'] > 0.5].sort_values(by="containment"))

# MAGs metadata
taxonomy = pd.read_excel("inputs/tully_mag_taxonomy.xlsx", header=1).set_index("Genome ID")
stats = pd.read_excel("inputs/tully_mag_stats.xlsx", header=1).set_index("Genome ID")

print("unique SRA runs: ", len(filtered['metagenome'].unique()))

                 MAG  metagenome  containment
89721     TOBG_NP-28  SRR2103020     0.500144
53860     TOBG_NP-42  SRR7168048     0.500169
28907    TOBG_EAC-55  SRR7479580     0.500182
73862     TOBG_SP-43  SRR7986296     0.500218
27250  TOBG_SAT-1356  SRR6713912     0.500264
...              ...         ...          ...
60804    TOBG_RS-626  ERR4013358     0.988004
66828    TOBG_SP-208  SRR5868539     0.989301
66807   TOBG_SP-4095  SRR5868539     0.990476
6268     TOBG_EAC-96  SRR8159436     0.992379
66794    TOBG_NP-110  SRR5868539     0.993755

[23644 rows x 3 columns]
unique SRA runs:  6398


In [4]:
len(filtered["MAG"].unique())

2291

In [5]:
len(filtered[filtered['containment'] > 0.5]["MAG"].unique())

1407

In [6]:
len(filtered[filtered['containment'] > 0.5]["metagenome"].unique())

2938

## Picking a candidate: TOBG_NP-110, I choose you!

In [112]:
taxonomy.loc['TOBG_NP-110']

Domain                           Archaea
Phylum                     Euryarchaeota
Class                                NaN
Order                                NaN
Family                               NaN
Genus                                NaN
Species                              NaN
Phylogeny source method           CheckM
Name: TOBG_NP-110, dtype: object

In [113]:
stats.loc['TOBG_NP-110']

Percent Complete                                                              70.27
Percent Contamination                                                          0.96
Strain Heterogeneity                                                             50
%G+C                                                                          56.83
No. of contigs                                                                   39
Max. contig length (bp)                                                      149808
N50 contig length (bp)                                                        44505
Mean contig length (bp)                                                       31750
Length (Mbp)                                                                   1.24
Coding density                                                                95.23
Data repository hosting the genome assembly reported in this study    NCBI Assembly
GenBank Accession ID for newly reported draft genomes                  PCBY0

In [117]:
with_link = filtered.copy()
with_link["metagenome"] = filtered["metagenome"].apply(
    lambda x: "<a href='https://trace.ncbi.nlm.nih.gov/Traces/sra/?run={}'>{}</a>".format(x, x)
)

HTML(with_link[(filtered["MAG"] == "TOBG_NP-110") & (filtered['containment'] > 0.5)]
     .sort_values(by="metagenome", ascending=False)
     .to_html(render_links=True, escape=False,))

,MAG,metagenome,containment
66684,TOBG_NP-110,SRR5868540,0.914910
66794,TOBG_NP-110,SRR5868539,0.993755
95736,TOBG_NP-110,SRR304680,0.640906
103789,TOBG_NP-110,SRR1509799,0.892272
104497,TOBG_NP-110,SRR1509798,0.984387
103893,TOBG_NP-110,SRR1509794,0.555816
103648,TOBG_NP-110,SRR1509793,0.787666
104606,TOBG_NP-110,SRR1509792,0.965652
92955,TOBG_NP-110,SRR070084,0.576893
93029,TOBG_NP-110,SRR070083,0.790008
